# MoodMorph Assessment Evaluation Analysis

Jonas de Araújo Luz Junior and
Maria Andréia Formico Rodrigues

PPGIA & Gira Lab, Universidade de Fortaleza (Unifor), 
Av. Washington Soares, 1321, Fortaleza, CE, Brazil, 60811-341.
___
# Qualitative Analysis Notebook

In [1]:
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt

from local.db_manager import DBManager

## Source data

The source data is read from the `Assessment.sqlite` SQLite database, which contains the prepared work data.

In [2]:
db = DBManager('data/Assessment.sqlite')

df_work = db.load_dataframe('work')
df_evaluations = db.load_dataframe('evaluations')

display(f"Index of df_work: {df_work.index.name}")
display(f"Columns of df_work: {df_work.columns.tolist()}")
display(f"Index of df_evaluations: {df_evaluations.index.name}")
display(f"Columns of df_evaluations: {df_evaluations.columns.tolist()}")


'Index of df_work: id'

"Columns of df_work: ['character_id', 'model', 'emotion', 'level', 'date_time', 'duration', 'animation', 'input_tokens', 'output_tokens', 'total_tokens', 'character_family', 'character_name', 'character_letter', 'model_name', 'calculated_tokens', 'delta_tokens', 'percent_delta_tokens', 'emotion_count']"

'Index of df_evaluations: id'

"Columns of df_evaluations: ['test_id', 'expected_emotion', 'identified_emotion', 'key_visual_cues', 'tokens_used', 'evaluator', 'timestamp']"

### Assessment Data Preparation

In [3]:
df_analysis = df_evaluations.join(df_work[['character_letter', 'model_name', 'emotion_count']], on='test_id')

df_analysis.head(2)

,test_id,expected_emotion,identified_emotion,key_visual_cues,tokens_used,evaluator,timestamp,character_letter,model_name,emotion_count
id,,,,,,,,,,
1,1,anger,"anger, disgust","Eyebrows are drawn down and together, eyes are...",694,openai/gpt-4.1,2025-10-24 14:25:49.352150,A,gpt-4o,1
2,2,contempt,contempt,"Slightly raised upper lip on one side, asymmet...",662,openai/gpt-4.1,2025-10-24 14:25:52.964229,A,gpt-4o,1


In [4]:
# Split emotions into tuples
#
split_and_sort = lambda s: tuple(sorted(s.split(' and ')))
is_expected = lambda expected, identified: all(
    str(emotion) in str(identified) for emotion in expected
)

df_analysis['expected'] = df_analysis['expected_emotion'].apply(split_and_sort)
df_analysis['identified'] = df_analysis['identified_emotion'].apply(split_and_sort)
df_analysis['verified'] = df_analysis.apply(
    lambda row: is_expected(row['expected'], row['identified']), axis=1
)

df_analysis[['expected_emotion', 'expected', 'identified_emotion', 'identified', 'verified']].head(10)

,expected_emotion,expected,identified_emotion,identified,verified
id,,,,,
1,anger,"(anger,)","anger, disgust","(anger, disgust,)",True
2,contempt,"(contempt,)",contempt,"(contempt,)",True
3,disgust,"(disgust,)",anger,"(anger,)",False
4,fear,"(fear,)","surprise, fear","(surprise, fear,)",True
5,happiness,"(happiness,)",happiness,"(happiness,)",True
6,sadness,"(sadness,)",sadness,"(sadness,)",True
7,surprise,"(surprise,)",surprise,"(surprise,)",True
8,happiness and surprise,"(happiness, surprise)",surprise,"(surprise,)",False
9,anger and contempt,"(anger, contempt)","anger, disgust","(anger, disgust,)",False


## Data Analysis

In [5]:
group = ['emotion_count', 'model_name', 'expected', 'character_letter']
grouped = df_analysis.groupby(group)

In [6]:
df_analysis.columns

Index(['test_id', 'expected_emotion', 'identified_emotion', 'key_visual_cues',
       'tokens_used', 'evaluator', 'timestamp', 'character_letter',
       'model_name', 'emotion_count', 'expected', 'identified', 'verified'],
      dtype='object')

In [7]:
mode_value = lambda x: x.mode().iloc[0] if not x.mode().empty else None
mode_count = lambda x: x.value_counts(sort=True).iloc[0] \
    if not x.value_counts().empty else 0
flat_tuple = lambda x: sum(x, ()) 

df_grouped = pd.DataFrame(grouped['identified'].agg([mode_value, mode_count, flat_tuple]))
df_grouped.rename(columns={
    '<lambda_0>':'identified_mode', 
    '<lambda_1>':'identified_mode_count', 
    '<lambda_2>':'all_identified'}, inplace=True)
df_grouped['consistency'] = df_grouped['identified_mode_count'] / grouped.size()

df_grouped.head(10)

identified_mode  \
emotion_count model_name       expected     character_letter                                
1             gemini-2.5-flash (anger,)     A                           (anger, disgust,)   
                                            M                                    (anger,)   
                               (contempt,)  A                                 (contempt,)   
                                            M                 (neutral, slight contempt,)   
                               (disgust,)   A                                    (anger,)   
                                            M                                (happiness,)   
                               (fear,)      A                           (fear, surprise,)   
                                            M                                 (surprise,)   
                               (happiness,) A                                (happiness,)   
                                            M                                (happiness,)   

                                                              identified_mode_count  \
emotion_count model_name       expected     character_letter                          
1             gemini-2.5-flash (anger,)     A                                    22   
                                            M                                    12   
                               (contempt,)  A                                    25   
                                            M                                    16   
                               (disgust,)   A                                    25   
                                            M                                    15   
                               (fear,)      A                                    21   
                                            M                                    25   
                               (happiness,) A                                    19   
                                            M                                    25   

                                                                                                 all_identified  \
emotion_count model_name       expected     character_letter                                                      
1             gemini-2.5-flash (anger,)     A                 (anger, disgust, anger, disgust, anger, disgus...   
                                            M                 (anger, anger, contempt, disgust, anger, anger...   
                               (contempt,)  A                 (contempt, contempt, contempt, contempt, conte...   
                                            M                 (neutral, slight contempt, neutral, slight con...   
                               (disgust,)   A                 (anger, anger, anger, anger, anger, anger, ang...   
                                            M                 (happiness, happiness, anger, contempt, happin...   
                               (fear,)      A                 (fear, surprise, fear, surprise, fear, surpris...   
                                            M                 (surprise, surprise, surprise, surprise, surpr...   
                               (happiness,) A                 (happiness, happiness, happiness, happiness, n...   
                                            M                 (happiness, happiness, happiness, happiness, h...   

                                                              consistency  
emotion_count model_name       expected     character_letter               
1             gemini-2.5-flash (anger,)     A                        0.88  
                                            M                        0.48  
                               (contempt,)  A                        1.00  
                                            M                        0.64  
                               (disgust,)   A                        1.00  

In [8]:
str_concat = lambda s: "|".join(sorted(set(s.apply(lambda x: x.strip()))))
unique_cues = lambda s: set(sum(s.apply(lambda x: x.split(',')), []))

df_cues = grouped['key_visual_cues'].apply(str_concat).reset_index().rename(
    columns={'key_visual_cues': 'visual_cues'}).set_index(group)

display(df_cues.head(10))

df_grouped = df_grouped.merge(df_cues, on=group)
df_grouped.head(10)

visual_cues
emotion_count model_name       expected     character_letter                                                   
1             gemini-2.5-flash (anger,)     A                 Brows are lowered and pulled together, eyes ar...
                                            M                 Brow is lowered and drawn together, eyes are n...
                               (contempt,)  A                 Asymmetrical mouth with one side slightly rais...
                                            M                 Face is mostly relaxed and symmetrical, with a...
                               (disgust,)   A                 Eyebrows are drawn down and together, creating...
                                            M                 Brows are drawn down and together in a frown, ...
                               (fear,)      A                 Wide open eyes with raised eyebrows, mouth ope...
                                            M                 Eyes are wide open, eyebrows are raised, and m...
                               (happiness,) A                 Mild upward curve of the mouth, relaxed eyes, ...
                                            M                 Mouth is open in a wide, symmetrical smile wit...

identified_mode  \
emotion_count model_name       expected     character_letter                                
1             gemini-2.5-flash (anger,)     A                           (anger, disgust,)   
                                            M                                    (anger,)   
                               (contempt,)  A                                 (contempt,)   
                                            M                 (neutral, slight contempt,)   
                               (disgust,)   A                                    (anger,)   
                                            M                                (happiness,)   
                               (fear,)      A                           (fear, surprise,)   
                                            M                                 (surprise,)   
                               (happiness,) A                                (happiness,)   
                                            M                                (happiness,)   

                                                              identified_mode_count  \
emotion_count model_name       expected     character_letter                          
1             gemini-2.5-flash (anger,)     A                                    22   
                                            M                                    12   
                               (contempt,)  A                                    25   
                                            M                                    16   
                               (disgust,)   A                                    25   
                                            M                                    15   
                               (fear,)      A                                    21   
                                            M                                    25   
                               (happiness,) A                                    19   
                                            M                                    25   

                                                                                                 all_identified  \
emotion_count model_name       expected     character_letter                                                      
1             gemini-2.5-flash (anger,)     A                 (anger, disgust, anger, disgust, anger, disgus...   
                                            M                 (anger, anger, contempt, disgust, anger, anger...   
                               (contempt,)  A                 (contempt, contempt, contempt, contempt, conte...   
                                            M                 (neutral, slight contempt, neutral, slight con...   
                               (disgust,)   A                 (anger, anger, anger, anger, anger, anger, ang...   
                                            M                 (happiness, happiness, anger, contempt, happin...   
                               (fear,)      A                 (fear, surprise, fear, surprise, fear, surpris...   
                                            M                 (surprise, surprise, surprise, surprise, surpr...   
                               (happiness,) A                 (happiness, happiness, happiness, happiness, n...   
                                            M                 (happiness, happiness, happiness, happiness, h...   

                                                              consistency  \
emotion_count model_name       expected     character_letter                
1             gemini-2.5-flash (anger,)     A                        0.88   
                                            M                        0.48   
                               (contempt,)  A                        1.00   
                                            M                        0.64   
                               (disgust,)   A                        

In [9]:
max5 = lambda s: s.nlargest(5).any()
df_top5 = grouped['verified'].apply(max5).reset_index().rename(
    columns={'verified': 'pass5'}
).set_index(group)

df_grouped = df_grouped.merge(df_top5, on=group)
df_grouped.head(10)

identified_mode  \
emotion_count model_name       expected     character_letter                                
1             gemini-2.5-flash (anger,)     A                           (anger, disgust,)   
                                            M                                    (anger,)   
                               (contempt,)  A                                 (contempt,)   
                                            M                 (neutral, slight contempt,)   
                               (disgust,)   A                                    (anger,)   
                                            M                                (happiness,)   
                               (fear,)      A                           (fear, surprise,)   
                                            M                                 (surprise,)   
                               (happiness,) A                                (happiness,)   
                                            M                                (happiness,)   

                                                              identified_mode_count  \
emotion_count model_name       expected     character_letter                          
1             gemini-2.5-flash (anger,)     A                                    22   
                                            M                                    12   
                               (contempt,)  A                                    25   
                                            M                                    16   
                               (disgust,)   A                                    25   
                                            M                                    15   
                               (fear,)      A                                    21   
                                            M                                    25   
                               (happiness,) A                                    19   
                                            M                                    25   

                                                                                                 all_identified  \
emotion_count model_name       expected     character_letter                                                      
1             gemini-2.5-flash (anger,)     A                 (anger, disgust, anger, disgust, anger, disgus...   
                                            M                 (anger, anger, contempt, disgust, anger, anger...   
                               (contempt,)  A                 (contempt, contempt, contempt, contempt, conte...   
                                            M                 (neutral, slight contempt, neutral, slight con...   
                               (disgust,)   A                 (anger, anger, anger, anger, anger, anger, ang...   
                                            M                 (happiness, happiness, anger, contempt, happin...   
                               (fear,)      A                 (fear, surprise, fear, surprise, fear, surpris...   
                                            M                 (surprise, surprise, surprise, surprise, surpr...   
                               (happiness,) A                 (happiness, happiness, happiness, happiness, n...   
                                            M                 (happiness, happiness, happiness, happiness, h...   

                                                              consistency  \
emotion_count model_name       expected     character_letter                
1             gemini-2.5-flash (anger,)     A                        0.88   
                                            M                        0.48   
                               (contempt,)  A                        1.00   
                                            M                        0.64   
                               (disgust,)   A                        

In [10]:
try:
    df_grouped.drop('all_identified', axis=1, inplace=True)
except KeyError:
    pass

df_grouped

identified_mode  \
emotion_count model_name       expected                       character_letter                                
1             gemini-2.5-flash (anger,)                       A                           (anger, disgust,)   
                                                              M                                    (anger,)   
                               (contempt,)                    A                                 (contempt,)   
                                                              M                 (neutral, slight contempt,)   
                               (disgust,)                     A                                    (anger,)   
...                                                                                                     ...   
3             grok-4-fast      (anger, contempt, disgust)     M                           (disgust, anger,)   
                               (anger, contempt, fear)        A                           (anger, disgust,)   
                                                              M                           (anger, disgust,)   
                               (happiness, sadness, surprise) A                                 (surprise,)   
                                                              M                                (happiness,)   

                                                                                identified_mode_count  \
emotion_count model_name       expected                       character_letter                          
1             gemini-2.5-flash (anger,)                       A                                    22   
                                                              M                                    12   
                               (contempt,)                    A                                    25   
                                                              M                                    16   
                               (disgust,)                     A                                    25   
...                                                                                               ...   
3             grok-4-fast      (anger, contempt, disgust)     M                                    13   
                               (anger, contempt, fear)        A                                    20   
                                                              M                                    13   
                               (happiness, sadness, surprise) A                                    10   
                                                              M                                    11   

                                                                                consistency  \
emotion_count model_name       expected                       character_letter                
1             gemini-2.5-flash (anger,)                       A                        0.88   
                                                              M                        0.48   
                               (contempt,)                    A                        1.00   
                                                              M                        0.64   
                               (disgust,)                     A                        1.00   
...                                                                                     ...   
3             grok-4-fast      (anger, contempt, disgust)     M                        0.52   
                               (anger, contempt, fear)        A                        0.80   
                                                              M                        0.52   
                               (happiness, sadness, surprise) A                        0.40   
                                                              M                        0.44   

                                              

In [11]:
df_grouped.to_excel('data/df_grouped.xlsx', sheet_name='Qualitative Analysis')